# Ultima versione 

Qui metto ultima versione.
Dovreste aggiungere spiegazione di ogni funzione e spiegazione iniziale.
In classe vi spiego

In [ ]:
import datetime
from time import sleep
from picamera import PiCamera
import os
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import scipy
#from skimage import img_as_float
import pandas as pd
import logging
import logzero
from logzero import logger
from sense_hat import SenseHat
   
sense = SenseHat()


# create a datetime variable to store the start time
start_time = datetime.datetime.now()
# create a datetime variable to store the current time
# (these will be almost the same at the start)
now_time = datetime.datetime.now()
print("tempo iniziale", start_time)
# Apro cattura immagini
camera = PiCamera()
camera.resolution = (1296,972)
# run a loop for 2 minutes

# formatting logfile csv
formatter = logging.Formatter('%(levelname)s, %(message)s');
logzero.formatter(formatter)


def write_line_on_csv(dir_path, val1, val2, val3, val4, val5):
    logzero.logfile(dir_path+"/data01.csv")
    print("writing logging csv file data01.csv")
    logzero.logger.info("%s, %s, %s, %s, %s", val1, val2, val3, val4, val5 )

        
def display_on_sensehat(stringa1, val1):
    blue = (0, 0, 255)
    yellow = (255, 255, 0)
    #sense.show_message("Astro Pi is awesome!", text_colour=yellow, back_colour=blue, scroll_speed=0.05)
    sense.show_message(stringa1, text_colour=yellow, back_colour=blue, scroll_speed=0.05)
    sleep(1)
    sense.show_message(str(val1), text_colour=yellow, back_colour=blue, scroll_speed=0.05)
    sleep(2)
    sense.clear()


def find_border(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0,0,0), (180,255,45))
    output = cv2.bitwise_and(img, img, mask=mask)
    return output

def find_sea(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (90,50,50), (125,255,255))
    output = cv2.bitwise_and(img, img, mask=mask)
    return output

def find_cloud(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (0,0,120), (180,30,255))
    output = cv2.bitwise_and(img, img, mask=mask)   
    return output

def find_ground(img, border, sea, cloud):
    output = cv2.bitwise_xor(img, border)
    output = cv2.bitwise_xor(output, sea)
    output = cv2.bitwise_xor(output, cloud)
    return output
    

def compute_percentage(masked_img):
    single_band_img= masked_img[:,:,0]>0
    npixel_class = sum(sum(1*single_band_img))
    return npixel_class
    

def global_classificator(img):
    border_output = find_border(img)
    sea_output = find_sea(img) 
    cloud_output = find_cloud(img)

    ground_output = find_ground(img, border_output, sea_output, cloud_output)
    nborder_pixel = compute_percentage(border_output)
    nsea_pixel = compute_percentage(sea_output)
    ncloud_pixel = compute_percentage(cloud_output)
    nground_pixel = compute_percentage(ground_output)
    total_img_pixel = img[:,:,0].shape[0]*img[:,:,0].shape[1]
    return [nborder_pixel, nsea_pixel, ncloud_pixel, nground_pixel, total_img_pixel]
    

counter = 0    

while (now_time < start_time + datetime.timedelta(minutes=3)):
    print("Doing stuff")
    sleep(1)
    

    dir_path = os.path.dirname(os.path.realpath(__file__))
    print(dir_path)
    f= open(dir_path+"/data01.csv","a")


    #camera.start_preview()
    # Camera warm-up time
    #sleep(2)
    print("capturing an image")
    filename='/image.jpg'
    path_and_filename = dir_path+filename
    camera.capture(path_and_filename)
    print("after image capture")
    img = cv2.imread(path_and_filename)
    nborder_pixel, nsea_pixel, ncloud_pixel, nground_pixel, total_img_pixel =global_classificator(img)
    print(" writing of csv file data01.csv")
    write_line_on_csv(dir_path,nborder_pixel, nsea_pixel, ncloud_pixel, nground_pixel, total_img_pixel)
    
    #write_line_on_csv(dir_path,npixel_i,pixel_totali)

    #scrivo sul senshat
    sea_percentual= round((nsea_pixel/(1.0*(total_img_pixel)))*100)
    border_percentual= round((nborder_pixel/(1.0*(total_img_pixel)))*100)
    
    print("sea_percentual", sea_percentual)
    display_on_sensehat('S% ',sea_percentual)
    sleep(1)
    display_on_sensehat('B% ',border_percentual)
    
    if (counter%10)==0:
        print("saving on cvs files ",counter )
        f.write('saving image number = '+str(counter)+'\n')
        image_rescaled = img[0:-1:4,0:-1:4,2]#rescale(bandR, 1.0 / 4.0, anti_aliasing=False)
        imgDataFrame= pd.DataFrame(image_rescaled)
        imgDataFrame.to_csv(dir_path+"/data01.csv",mode='a', index=False)
        f.write('saved image number = '+str(counter)+'\n')
    
    #rimuovo immagine
    os.remove(dir_path+'/image.jpg')
    # update the current time
    now_time = datetime.datetime.now()
    print("tempo attuale", now_time, " delta_tempo", datetime.timedelta(minutes=2))
    counter = counter+1
    print("counter = ", counter) 

f.close()
sense.clear()


# Testare classificazione dati

In [1]:
!pip install opencv-python

    100% |████████████████████████████████| 25.4MB 8.3kB/s eta 0:00:01 2% |▉                               | 696kB 8.4MB/s eta 0:00:03    4% |█▌                              | 1.2MB 2.7MB/s eta 0:00:09    10% |███▌                            | 2.8MB 1.3MB/s eta 0:00:19    11% |███▌                            | 2.8MB 922kB/s eta 0:00:25    12% |███▉                            | 3.1MB 1.7MB/s eta 0:00:14    13% |████▎                           | 3.4MB 1.8MB/s eta 0:00:13    29% |█████████▍                      | 7.4MB 2.3MB/s eta 0:00:08    31% |██████████▏                     | 8.1MB 1.9MB/s eta 0:00:09    39% |████████████▊                   | 10.1MB 718kB/s eta 0:00:22    39% |████████████▊                   | 10.1MB 322kB/s eta 0:00:48    45% |██████████████▌                 | 11.5MB 961kB/s eta 0:00:15    66% |█████████████████████▍          | 17.0MB 1.2MB/s eta 0:00:08    74% |███████████████████████▉        | 18.9MB 1.2MB/s eta 0:00:06


In [ ]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import scipy 

from skimage.morphology import erosion
from skimage.morphology import binary_dilation
from skimage.color import rgb2gray

def mean_class_value(img, mask):
    tmp = img*mask
    flat_img = tmp.flat
    flat_mask= mask.flat
    
    sum_values = sum(flat_img)
    npixel = sum(flat_mask)
    mean_value = sum_values/(1.0*npixel)
    return mean_value

def find_contour(grayscale):
    """ this function gives in input a grayscale image and returns a mask 
    image with True or False values. True if pixel i,j belongs to class contour
    and False otherwise"""
    
    blur_kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])/9.0;
    grayscale = scipy.signal.convolve2d(grayscale, blur_kernel, 'same') #to check 
    
    grayscale = erosion(grayscale) #to check
    bordo1= grayscale[0:500,0:20]
    bordo2 = grayscale[0:500,:-20:-1]
    mean_of_borders = (bordo1.mean()+bordo2.mean())/2.0
    mask1= grayscale<( mean_of_borders+9*bordo1.std())


    mask1= binary_dilation(mask1)
    mask1= binary_dilation(1*mask1)    
    return mask1


def std_image(grayscale):
    blur_kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])/9.0;
    meank = scipy.signal.convolve2d(grayscale, blur_kernel, 'same')
    
    tmp = grayscale -meank
    tmp = tmp*tmp
    std_image = scipy.signal.convolve2d(tmp, blur_kernel, 'valid')
    std_image = scipy.sqrt(std_image)
    
    return std_image

#def mean_maskedimage(bandimage, mask):
#    tmp = bandimage*(1-mask)
#    sum_of_values_ =tmp.sum()
#    
#    tot_pixel =  float(band1.shape[0]*band1.shape[0]-mask2.sum())
#    mean_maskedimage = sum_of_values/tot_pixel
#    
#    return mean_maskedimage
    

def find_cloud(img):
    bandR = img_t[:,:,0]
    bandG = img_t[:,:,1]
    bandB = img_t[:,:,2]
    nline = bandR.shape[0]
    ncolumn = bandR.shape[1]  ### da cambiare
    #bandR_subset = bandR[round(nline/4):round((nline/4)+(nline/2)),round(nline/4):round((nline/4)+(nline/2))]
    
    mean_class_value(img, mask)
    bandR_subset = bandR
    mask1 = bandR>bandR_subset.mean()+1.3*bandR_subset.std()#classe nuvola
    return mask1


def find_cloud2(img, mask_contour):
    bandR = img_t[:,:,0]
    bandG = img_t[:,:,1]
    bandB = img_t[:,:,2]
    nline = bandR.shape[0]
    ncolumn = bandR.shape[1]  ### da cambiare
    #bandR_subset = bandR[round(nline/4):round((nline/4)+(nline/2)),round(nline/4):round((nline/4)+(nline/2))]
    
    mean_circle = mean_class_value(bandR, mask_contour)
    bandR_subset = bandR
    mask1 = bandR>mean_circle+1.3*bandR_subset.std()#classe nuvola
    return mask1


def find_sea(img, mask_contour):
    bandR = img[:,:,0]
    bandG = img[:,:,1]
    bandB = img[:,:,2]
    #grayscale = rgb2gray(img)
    
    mask_sea = (bandB<bandB.mean())*(bandB>10) #mare
    mask_sea = mask_sea*(1-mask_contour)
    return mask_sea


def find_sea2(img, mask_contour):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (90,50,0), (122,255,255))
    output = cv2.bitwise_and(img, img, mask=mask)
    #ratio = cv2.countNonZero(mask)/(img.size/3)
    return mask

def find_ground(img, mask_sea,mask_cloud):
    bandR = img[:,:,0]
    bandG = img[:,:,1]
    bandB = img[:,:,2]
    
    nline = bandR.shape[0]
    ncolumn = bandR.shape[1]  ### da cambiare
    start_line = round(nline/4)
    end_line = round(nline/4) +round(nline/2)
    start_column = round(ncolumn/4)
    end_column = round(ncolumn/4) +round(ncolumn/2)
   
    bandR_subset = bandR[start_line:end_line,start_column:end_column]
    mask3 = bandR> (bandR_subset.mean()+0.2*bandR_subset.std())
    mask3= mask3*(1-mask_sea)*(1-mask_cloud)
    return mask3

def compute_pixel_class(mask):
    #ntotal_pixel = mask.shape[0]*mask.shape[1]
    tmp_px = np.where(mask==True)
    npixel_class=len(tmp_px[0])
    return npixel_class



if __name__ == "__main__":
    import glob
    
    
    #list_of_filename= [ 'mista1.jpg', 'mista2.png','mista3.png','mista4.png']
    path ='./data/'
    list_of_filename = glob.glob(path+'*.png')
    counter = 10
    
    for i in list_of_filename:
        
        #path_and_filename = path+i
        print(i)
        img_t = cv2.imread(i)
        img_t = img_t[:, :, ::-1]
        bandR = img_t[:,:,0]
        bandG = img_t[:,:,1]
        bandB = img_t[:,:,2]
        
        
        grayscale = rgb2gray(img_t)
        plt.figure(1)
        plt.subplot(2,3,1)
        plt.imshow(img_t)
        plt.title('originale '+i[7:-4])
        print(i,"gray mean=",grayscale.mean(), " gray std=", grayscale.std(),"\n")
        print(i,"bandR mean=", bandR.mean(), " bandG mean=", bandG.mean(), "BandB mean=", \
              bandB.mean(),"\n")
        print(i,"bandR std=", bandR.std(), " bandG std=", bandG.std(), "BandB std=", \
              bandB.std(),"\n")
        class_contour =find_contour(grayscale)
        contour_mean = mean_class_value(bandR, class_contour)
        #from skimage.morphology import binary_closing
        #final = binary_closing(class_mask)
        plt.subplot(2,3,2)
        plt.imshow(class_contour)
        plt.title('   border'+str(round(contour_mean)))
        #class_cloud = find_cloud(img_t)
        class_cloud = find_cloud2(img_t,class_contour)
        cloud_mean = mean_class_value(bandR, class_cloud)
        plt.subplot(2,3,3)
        plt.imshow(class_cloud)
        plt.title(' cloud '+str(round(cloud_mean)))
        
        class_sea = find_sea(img_t,class_contour)
        sea_mean = mean_class_value(bandR, class_sea)
        plt.subplot(2,3,4)
        plt.imshow(class_sea)
        plt.title(' sea'+str(round(sea_mean)))
        
        
        class_ground = find_ground(img_t,class_sea,class_cloud)
        plt.subplot(2,3,5)
        plt.imshow(class_ground)
        ground_mean = mean_class_value(bandR, class_ground)
        plt.title(' ground'+str(round(ground_mean)))
        
        counter = counter+1
        npixel_total = bandR.shape[0]*bandR.shape[1]
        npixel_contour = compute_pixel_class(class_contour)
        npixel_cloud = compute_pixel_class(class_cloud)
        npixel_sea = compute_pixel_class(class_sea)
        npixel_ground = compute_pixel_class(class_ground)
        
        
        print("pixel delle classi: ",npixel_total," contour ", npixel_contour," cloud ",npixel_cloud, " sea ", npixel_sea, " ground ", npixel_ground)
        
        final_classified_image = class_contour+ 4*class_cloud+3*class_sea+2*class_ground
        #final_classified_image = class_contour*class_cloud*class_sea*class_ground
        
        print("max final_imnage",final_classified_image.max())
        
        plt.subplot(2,3,6)
        plt.imshow(final_classified_image,cmap='tab10')
        plt.title(' image with all classes')
        
        plt.show()

# Funzione che classifica le immagini

- Dovrete realizzare un classificatore per le immagini prese dal raspberry delle seguenti classi:

- gruppo1 classifica mare
- gruppo2: nuvole
-gruppo 3: terra
- gruppo4: misto

dataset con  4 cartelle contenenti ognuna esempi di ogni gruppo.



In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 14 22:26:05 2019

@author: davpas
"""

import sys
import cv2 
import numpy as np
import matplotlib.pyplot as plt

from skimage import img_as_float

path = './foto/'
filename = 'foto1.jpg'
print("argomento in input", sys.argv[1])
filename = sys.argv[1]

path_and_filename = path+filename

img = cv2.imread(path_and_filename)
img = img[:, :, ::-1]
imagescimage = img_as_float(img)

print(img)
print(type(img))

print(len(img))
band1= img[:,:,0]
band2 = img[:,:,1]
band3 = img[:,:,2]
print(dir(band1))
print(band1.size)
plt.figure(1)
plt.imshow(band1, cmap=plt.cm.gray)
plt.show()
plt.figure(2)
plt.imshow(band1, cmap=plt.cm.gray)
plt.show()

plt.figure(3)
plt.imshow(band1, cmap=plt.cm.gray)
plt.figure(4)
plt.imshow(img)
plt.show()
print(band1.mean(),band1.std())
treshold = 100
mask1 = band1>band1.mean()
plt.figure(5)
#img_mask = np.int(mask1)
#plt.plot(img_mask)
plt.imshow(mask1, cmap=plt.cm.gray)
plt.show()
print(sum(mask1))


# Cosa fare durante vacanze

1) Leggere guida fase 2

2) Vedere Video di aiuto Team Astro Pi 

3) Installare OpenCV sul proprio notebook 

4) Trovare le immagini su cui fare i test


5) Esercitarsi sul tutorial OpenCV

6) Esercitarsi sullo scrivere i testi sulla matrice LED 

7) Esercitarsi sul salvare su file in formato csv dei valori 




Qui di seguito per ognuno dei punti metto spiegazione

---------------------------------------------------------------------------------



# Punto 1: Guida fase 2

La guida alla fase due la trovate al seguente link:
https://projects.raspberrypi.org/en/projects/code-for-your-astro-pi-mission-space-lab-experiment/2

Leggetela passo passo e prendete appunti di questa guida.
Non dovete capire passo passo tutto, vi chiedo però di inviarmi una email con  domande delle cose che non vi sono chiare della guida.

Aspetto questa vostra e-mail entro il 28 dicembre



# Punto 2: Vedere video tutorial di aiuto Team Astro Pi

Questo compito è semplice:  dovete vedere su youtube il video di supporto presente al seguente link:

https://www.youtube.com/watch?v=pvHW0tsFafI

Prendete appunti delle parti che vi sembrano importanti. 



#  Punto 3: Installare OpenCV sul vostro notebook 

- Create un vostro notebook azure dedicato
- Installate la libreria OpenCV usando il seguente comando: !pip install opencv-python

Qui di seguito è mostrato cosa succede.



In [2]:
!pip install opencv-python

    100% |████████████████████████████████| 25.4MB 150kB/s eta 0:00:01    50% |████████████████▏               | 12.8MB 10.4MB/s eta 0:00:02


#  Punto 4: trovate immagini su cui fare test

Vi invito a cercare le immagini su cui fare i test. 
Dovrebbero essre in internet (ne parlano del video del Team AstroPi)

Il primo che le trova invia il link a tutti



# Punto 5: esercitarsi su tutorial OpenCV

Importante: oltre ad installare OpenCV sul vostro notebook sarebbe utile installarlo anche sul vostro computer

Vi invito a leggere il tutorial scritto qui: 
https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_tutorials.html



In particolare: 

https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_image_display/py_image_display.html#py-display-image


https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_core/py_basic_ops/py_basic_ops.html#basic-ops



## Esercizio da fare :
create un notebook in cui 

- Aprite una immagine

- Leggete i valori di alcuni pixel di una banda di questa immagine.
vi ricordo che le immagini hanno 3 bande: r, g, b (rossa, verde e blu)


- Calcolate media di questi valori

Inviatemi link al notebook con esercizio svolto


Suggerimenti: per mostrare immagine usate per esempio questo codice:  

import numpy as np
import cv2
from matplotlib import pyplot as plt

img = cv2.imread('messi5.jpg',0)
plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
plt.show()


Per accedere ai pixel: 

>>> import cv2
>>> import numpy as np

>>> img = cv2.imread('messi5.jpg')
You can access a pixel value by its row and column coordinates. For BGR image, it returns an array of Blue, Green, Red values. For grayscale image, just corresponding intensity is returned.

>>> px = img[100,100]
>>> print px
[157 166 200]

Accedere solo ai pixel blu
>>> blue = img[100,100,0]




# Punto 6: esercitarsi sullo scrivere i testi sulla matrice LED

Qui trovate il link all'emulatore di sense-hat ovvero quell'oggetto che aggiunto al raspberry lo dota di sensori e di una matrici di led.

Vedere qui: 
https://www.raspberrypi.org/blog/sense-hat-emulator/

e soprattutto qui: 
https://trinket.io/sense-hat

Esercizio: 
inviatemi link a notebook in cui scrivete una frase tipo "AstroRussell" sulla matrice di Led



In [1]:
# DA qui in poi non leggere... da finire

import cv2

ModuleNotFoundError: No module named 'cv2'

# Link Utili
- Vedere come accedere a telecamera:
https://www.pyimagesearch.com/2015/03/30/accessing-the-raspberry-pi-camera-with-opencv-and-python/
